<a href="https://www.kaggle.com/code/loubl00m/llama-chatbot?scriptVersionId=274312260" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Loading modules

In [1]:
# Install the necessary libraries
!pip install -U torch==2.4.0 transformers==4.46.2 datasets==3.1.0 accelerate==1.1.1 \
    peft==0.13.2 trl==0.12.1 bitsandbytes==0.46.1
!pip install --no-cache-dir wandb==0.17.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from huggingface_hub import login, Repository, create_repo
from trl import SFTConfig, SFTTrainer
from kaggle_secrets import UserSecretsClient
import wandb

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-11-07 17:29:34.063000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-07 17:29:34.063138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-07 17:29:34.199090: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

In [3]:
# Get the Weights & Biases keys from env and login
user_secrets = UserSecretsClient()
wb_key = user_secrets.get_secret("wandb_key")
wandb.login(key = wb_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# Get my hugging face token for llama
hf_token = user_secrets.get_secret("hf_token")
login(token = hf_token, add_to_git_credential=True)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading the model

In [5]:
# Choosing the attention implementation according to the version of the GPU if it exists.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [6]:
# QLoRA config with normal float 4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config, adding to the linear (matrix multiply) layers
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [7]:
# Instruct is already fine-tuned for instructions rather than completion.
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [8]:
# Adding quantization and LoRA to the model
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Preparing the dataset

### The dataset has been shuffled, split and set to the correct format in advance.

In [9]:
train_path = "/kaggle/input/student-assistance/train.jsonl"
val_path = "/kaggle/input/student-assistance/test.jsonl"
# One split for each because they're separated in advance
train_set = load_dataset("json", data_files = train_path, split="train")
val_set = load_dataset("json", data_files = val_path, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

# Model training

In [10]:
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

0

In [11]:
# Supervised finetuning configuration
lr = 1e-5
batch_size = 8
max_seq_length = 512
epochs = 10
seed = 42
repo_name = "llama-edu"

training_args = SFTConfig(
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        gradient_accumulation_steps = 2,
        eval_steps = 20,
        warmup_steps = 20,
        num_train_epochs = epochs,
        learning_rate = lr,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = seed,
        max_seq_length = max_seq_length,
        dataset_num_proc = 2,
        run_name = "llama_finetuning",
        output_dir = repo_name,
        eval_strategy = "steps",
        logging_strategy = "steps",
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = val_set,
    #packing = False, # Can make training 5x faster for short sequences.
    args = training_args,
    peft_config = peft_config,
)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/310 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
# Evaluation before finetuning.
trainer.evaluate()

wandb: Currently logged in as: louay-yahyaoui (louay-yahyaoui-insat). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251107_173246-0ymb4j53
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run llama_finetuning
wandb: ⭐️ View project at https://wandb.ai/louay-yahyaoui-insat/huggingface
wandb: 🚀 View run at https://wandb.ai/louay-yahyaoui-insat/huggingface/runs/0ymb4j53


{'eval_loss': 2.8933908939361572,
 'eval_model_preparation_time': 0.0114,
 'eval_runtime': 57.4151,
 'eval_samples_per_second': 1.742,
 'eval_steps_per_second': 0.226}

In [13]:
# Training for 10 epochs.
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Model Preparation Time
20,3.444600,2.116335,0.011400
40,1.265500,1.228370,0.011400
60,0.971300,0.960500,0.011400
80,0.798500,0.873541,0.011400
100,0.742200,0.839555,0.011400
120,0.799200,0.819823,0.011400
140,0.801500,0.806334,0.011400
160,0.780600,0.798623,0.011400
180,0.743600,0.794970,0.011400


TrainOutput(global_step=190, training_loss=1.130711114092877, metrics={'train_runtime': 5700.7074, 'train_samples_per_second': 0.544, 'train_steps_per_second': 0.033, 'total_flos': 2.4531002935443456e+16, 'train_loss': 1.130711114092877, 'epoch': 9.743589743589745})

In [14]:
# Post-training evaluation
trainer.evaluate()

{'eval_loss': 0.794126033782959,
 'eval_model_preparation_time': 0.0114,
 'eval_runtime': 56.682,
 'eval_samples_per_second': 1.764,
 'eval_steps_per_second': 0.229,
 'epoch': 9.743589743589745}

In [15]:
# Storing the trained model in hugging face.
trainer.push_to_hub("add: llama for education")

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1762542541.4603311388dd.24.1:   0%|          | 0.00/425 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

events.out.tfevents.1762536766.4603311388dd.24.0:   0%|          | 0.00/49.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LouayYahyaoui/llama-edu/commit/666087d3d3facb3f0bcd91346ed216a64eb0db03', commit_message='add: llama for education', commit_description='', oid='666087d3d3facb3f0bcd91346ed216a64eb0db03', pr_url=None, pr_revision=None, pr_num=None)